# GANs

In [ ]:
%matplotlib inline
from fastai.gen_doc.nbdoc import *
from fastai.vision import * 
from fastai.vision.gan import *

GAN stands for [Generative Adversarial Nets](https://arxiv.org/pdf/1406.2661.pdf) and were invented by Ian Goodfellow. The concept is that we will train two models at the same time: a generator and a critic. The generator will try to make new images similar to the ones in our dataset, and the critic's job will try to classify real images from the fake ones the generator does. The generator returns images, the discriminator a feature map (it can be a single number depending on the input size). Usually the discriminator will be trained to retun 0. everywhere for fake images and 1. everywhere for real ones.

This module contains all the necessary function to create a GAN.

We train them against each other in the sense that at each step (more or less), we:
1. Freeze the generator and train the discriminator for one step by:
  - getting one batch of true images (let's call that `real`)
  - generating one batch of fake images (let's call that `fake`)
  - have the discriminator evaluate each batch and compute a loss function from that; the important part is that it rewards positively the detection of real images and penalizes the fake ones
  - update the weights of the discriminator with the gradients of this loss
  
  
2. Freeze the discriminator and train the generator for one step by:
  - generating one batch of fake images
  - evaluate the discriminator on it
  - return a loss that rewards posisitivly the discriminator thinking those are real images; the important part is that it rewards positively the detection of real images and penalizes the fake ones
  - update the weights of the generator with the gradients of this loss

In [ ]:
show_doc(GANLearner)

<h2 id="GANLearner"><code>class</code> <code>GANLearner</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L204" class="source_link">[source]</a></h2>

> <code>GANLearner</code>(<b>`data`</b>:[`DataBunch`](/basic_data.html#DataBunch), <b>`generator`</b>:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), <b>`critic`</b>:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), <b>`gen_loss_func`</b>:`LossFunction`, <b>`crit_loss_func`</b>:`LossFunction`, <b>`switcher`</b>:[`Callback`](/callback.html#Callback)=<b><i>`None`</i></b>, <b>`gen_first`</b>:`bool`=<b><i>`False`</i></b>, <b>`switch_eval`</b>:`bool`=<b><i>`True`</i></b>, <b>`show_img`</b>:`bool`=<b><i>`True`</i></b>, <b>`clip`</b>:`float`=<b><i>`None`</i></b>, <b>`kwargs`</b>) :: [`Learner`](/basic_train.html#Learner)

A [`Learner`](/basic_train.html#Learner) suitable for GANs.  

This is the general constructor to create a GAN, you might want to use one of the factory methods that are easier to use. Create a GAN from [`data`](/vision.data.html#vision.data), a `generator` and a `critic`. The [`data`](/vision.data.html#vision.data) should have the inputs the `generator` will expect and the images wanted as targets.

`gen_loss_func` is the loss function that will be applied to the `generator`. It takes three argument `fake_pred`, `target`, `output` and should return a rank 0 tensor. `output` is the result of the `generator` applied to the input (the xs of the batch), `target` is the ys of the batch and `fake_pred` is the result of the `discriminator` being given `output`. `output`and `target` can be used to add a specific loss to the GAN loss (pixel loss, feature loss) and for a good training of the gan, the loss should encourage `fake_pred` to be as close to 1 as possible (the `generator` is trained to fool the `critic`).

`crit_loss_func` is the loss function that will be applied to the `critic`. It takes two arguments `real_pred` and `fake_pred`. `real_pred` is the result of the `critic` on the target images (the ys of the batch) and `fake_pred` is the result of the `critic` applied on a batch of fake, generated byt the `generator` from the xs of the batch.

`switcher` is a [`Callback`](/callback.html#Callback) that should tell the GAN when to switch from critic to generator and vice versa. By default it does 5 iterations of the critic for 1 iteration of the generator. The model begins the training with the `generator` if `gen_first=True`. If `switch_eval=True`, the model that isn't trained is switched on eval mode (left in training mode otherwise, which means some statistics like the running mean in batchnorm layers are updated, or the dropouts are applied).

`clip` should be set to a certain value if one wants to clip the weights (see the [Wassertein GAN](https://arxiv.org/pdf/1701.07875.pdf) for instance).

If `show_img=True`, one image generated by the GAN is shown at the end of each epoch.

### Factory methods

In [ ]:
show_doc(GANLearner.from_learners)

<h4 id="GANLearner.from_learners"><code>from_learners</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L217" class="source_link">[source]</a></h4>

> <code>from_learners</code>(<b>`learn_gen`</b>:[`Learner`](/basic_train.html#Learner), <b>`learn_crit`</b>:[`Learner`](/basic_train.html#Learner), <b>`switcher`</b>:[`Callback`](/callback.html#Callback)=<b><i>`None`</i></b>, <b>`weights_gen`</b>:`Point`=<b><i>`None`</i></b>, <b>`kwargs`</b>)

Create a GAN from `learn_gen` and `learn_crit`.  

Directly creates a [`GANLearner`](/vision.gan.html#GANLearner) from two [`Learner`](/basic_train.html#Learner): one for the `generator` and one for the `critic`. The `switcher` and all `kwargs` will be passed to the initialization of [`GANLearner`](/vision.gan.html#GANLearner) along with the following loss functions:

- `loss_func_crit` is the mean of `learn_crit.loss_func` applied to `real_pred` and a target of ones with `learn_crit.loss_func` applied to `fake_pred` and a target of zeros
- `loss_func_gen` is the mean of `learn_crit.loss_func` applied to `fake_pred` and a target of ones (to full the discriminator) with `learn_gen.loss_func` applied to `output` and `target`. The weights of each of those contributions can be passed in `weights_gen` (default is 1. and 1.)

In [ ]:
show_doc(GANLearner.wgan)

<h4 id="GANLearner.wgan"><code>wgan</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L224" class="source_link">[source]</a></h4>

> <code>wgan</code>(<b>`data`</b>:[`DataBunch`](/basic_data.html#DataBunch), <b>`generator`</b>:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), <b>`critic`</b>:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), <b>`switcher`</b>:[`Callback`](/callback.html#Callback)=<b><i>`None`</i></b>, <b>`clip`</b>:`float`=<b><i>`0.01`</i></b>, <b>`kwargs`</b>)

Create a WGAN from `data`, `generator` and `critic`.  

The Wasserstein GAN is detailed in [this article]. `switcher` and the `kwargs` will be passed to the [`GANLearner`](/vision.gan.html#GANLearner) init, `clip`is the weight clipping.

## Switchers

In any GAN training, you will need to tell the [`Learner`](/basic_train.html#Learner) when to switch from generator to critic and vice versa. The two following [`Callback`](/callback.html#Callback) are examples to help you with that.

As usual, don't call the `on_something` methods directly, the fastai library will do it for you during training.

In [ ]:
show_doc(FixedGANSwitcher, title_level=3)

<h3 id="FixedGANSwitcher"><code>class</code> <code>FixedGANSwitcher</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L153" class="source_link">[source]</a></h3>

> <code>FixedGANSwitcher</code>(<b>`learn`</b>:[`Learner`](/basic_train.html#Learner), <b>`n_crit`</b>:`Union`\[`int`, `Callable`\]=<b><i>`1`</i></b>, <b>`n_gen`</b>:`Union`\[`int`, `Callable`\]=<b><i>`1`</i></b>) :: [`LearnerCallback`](/basic_train.html#LearnerCallback)

Switcher to do `n_crit` iterations of the critic then `n_gen` iterations of the generator.  

In [ ]:
show_doc(FixedGANSwitcher.on_train_begin)

<h4 id="FixedGANSwitcher.on_train_begin"><code>on_train_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L158" class="source_link">[source]</a></h4>

> <code>on_train_begin</code>(<b>`kwargs`</b>)

Initiate the iteration counts.  

In [ ]:
show_doc(FixedGANSwitcher.on_batch_end)

<h4 id="FixedGANSwitcher.on_batch_end"><code>on_batch_end</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L162" class="source_link">[source]</a></h4>

> <code>on_batch_end</code>(<b>`iteration`</b>, <b>`kwargs`</b>)

Switch the model if necessary.  

In [ ]:
show_doc(AdaptiveGANSwitcher, title_level=3)

<h3 id="AdaptiveGANSwitcher"><code>class</code> <code>AdaptiveGANSwitcher</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L176" class="source_link">[source]</a></h3>

> <code>AdaptiveGANSwitcher</code>(<b>`learn`</b>:[`Learner`](/basic_train.html#Learner), <b>`gen_thresh`</b>:`float`=<b><i>`None`</i></b>, <b>`critic_thresh`</b>:`float`=<b><i>`None`</i></b>) :: [`LearnerCallback`](/basic_train.html#LearnerCallback)

Switcher that goes back to generator/critic when the loes goes below `gen_thresh`/`crit_thresh`.  

In [ ]:
show_doc(AdaptiveGANSwitcher.on_batch_end)

<h4 id="AdaptiveGANSwitcher.on_batch_end"><code>on_batch_end</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L181" class="source_link">[source]</a></h4>

> <code>on_batch_end</code>(<b>`last_loss`</b>, <b>`kwargs`</b>)

Switch the model if necessary.  

## Discriminative LR

If you want to train your critic at a different learning rate than the generator, this will let you do it automatically (even if you have a learning rate schedule).

In [ ]:
show_doc(GANDiscriminativeLR, title_level=3)

<h3 id="GANDiscriminativeLR"><code>class</code> <code>GANDiscriminativeLR</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L278" class="source_link">[source]</a></h3>

> <code>GANDiscriminativeLR</code>(<b>`learn`</b>:[`Learner`](/basic_train.html#Learner), <b>`mult_lr`</b>:`float`=<b><i>`5.0`</i></b>) :: [`LearnerCallback`](/basic_train.html#LearnerCallback)

[`Callback`](/callback.html#Callback) that handles multiplying the learning rate by `mult_lr` for the critic.  

In [ ]:
show_doc(GANDiscriminativeLR.on_batch_begin)

<h4 id="GANDiscriminativeLR.on_batch_begin"><code>on_batch_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L282" class="source_link">[source]</a></h4>

> <code>on_batch_begin</code>(<b>`train`</b>, <b>`kwargs`</b>)

Multiply the current lr if necessary.  

In [ ]:
show_doc(GANDiscriminativeLR.on_step_end)

<h4 id="GANDiscriminativeLR.on_step_end"><code>on_step_end</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L286" class="source_link">[source]</a></h4>

> <code>on_step_end</code>(<b>`kwargs`</b>)

Put the LR back to its value if necessary.  

## Specific models

In [ ]:
show_doc(basic_critic)

<h4 id="basic_critic"><code>basic_critic</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L17" class="source_link">[source]</a></h4>

> <code>basic_critic</code>(<b>`in_size`</b>:`int`, <b>`n_channels`</b>:`int`, <b>`n_features`</b>:`int`=<b><i>`64`</i></b>, <b>`n_extra_layers`</b>:`int`=<b><i>`0`</i></b>, <b>`kwargs`</b>)

A basic critic for images `n_channels` x `in_size` x `in_size`.  

This model contains a first 4 by 4 convolutional layer of stride 2 from `n_channels` to `n_features` followed by `n_extra_layers` 3 by 3 convolutional layer of stride 1. Then we put as many 4 by 4 convolutional layer of stride 2 with a number of features multiplied by 2 at each stage so that the `in_size` becomes 1. `kwargs` can be used to customize the convolutional layers and are passed to [`conv_layer`](/layers.html#conv_layer).

In [ ]:
show_doc(basic_generator)

<h4 id="basic_generator"><code>basic_generator</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L28" class="source_link">[source]</a></h4>

> <code>basic_generator</code>(<b>`in_size`</b>:`int`, <b>`n_channels`</b>:`int`, <b>`noise_sz`</b>:`int`=<b><i>`100`</i></b>, <b>`n_features`</b>:`int`=<b><i>`64`</i></b>, <b>`n_extra_layers`</b>=<b><i>`0`</i></b>, <b>`kwargs`</b>)

A basic generator from `noise_sz` to images `n_channels` x `in_size` x `in_size`.  

This model contains a first 4 by 4 transposed convolutional layer of stride 1 from `noise_size` to the last numbers of features of the corresponding critic. Then we put as many 4 by 4 transposed convolutional layer of stride 2 with a number of features divided by 2 at each stage so that the image ends up being of height and widht `in_size//2`. At the end, we add`n_extra_layers` 3 by 3 convolutional layer of stride 1. The last layer is a transpose convolution of size 4 by 4 and stride 2 followed by `tanh`. `kwargs` can be used to customize the convolutional layers and are passed to [`conv_layer`](/layers.html#conv_layer).

In [ ]:
show_doc(gan_critic)

<h4 id="gan_critic"><code>gan_critic</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L260" class="source_link">[source]</a></h4>

> <code>gan_critic</code>(<b>`n_channels`</b>:`int`=<b><i>`3`</i></b>, <b>`nf`</b>:`int`=<b><i>`128`</i></b>, <b>`n_blocks`</b>:`int`=<b><i>`3`</i></b>, <b>`p`</b>:`int`=<b><i>`0.15`</i></b>)

Critic to train a `GAN`.  

In [ ]:
show_doc(GANTrainer)

<h2 id="GANTrainer"><code>class</code> <code>GANTrainer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L72" class="source_link">[source]</a></h2>

> <code>GANTrainer</code>(<b>`learn`</b>:[`Learner`](/basic_train.html#Learner), <b>`switch_eval`</b>:`bool`=<b><i>`False`</i></b>, <b>`clip`</b>:`float`=<b><i>`None`</i></b>, <b>`beta`</b>:`float`=<b><i>`0.98`</i></b>, <b>`gen_first`</b>:`bool`=<b><i>`False`</i></b>, <b>`show_img`</b>:`bool`=<b><i>`True`</i></b>) :: [`LearnerCallback`](/basic_train.html#LearnerCallback)

Handles GAN Training.  

[`LearnerCallback`](/basic_train.html#LearnerCallback) that will be responsible to handle the two different optimizers (one for the generator and one for the critic), and do all the work behind the scenes so that the generator (or the critic) are in training mode with parameters requirement gradients each time we switch.

`switch_eval=True` means that the [`GANTrainer`](/vision.gan.html#GANTrainer) will put the model that isn't training into eval mode (if it's `False` its running statistics like in batchnorm layers will be updated and dropout will be applied). `clip` is the clipping applied to the weights (if not `None`). `beta` is the coefficient for the moving averages as the [`GANTrainer`](/vision.gan.html#GANTrainer)tracks separately the generator loss and the critic loss. `gen_first=True` means the training begins with the generator (with the critic if it's `False`). If `show_img=True` we show a generated image at the end of each epoch.

In [ ]:
show_doc(GANTrainer.switch)

<h4 id="GANTrainer.switch"><code>switch</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L144" class="source_link">[source]</a></h4>

> <code>switch</code>(<b>`gen_mode`</b>:`bool`=<b><i>`None`</i></b>)

Switch the model, if `gen_mode` is provided, in the desired mode.  

If `gen_mode` is left as `None`, just put the model in the other mode (critic if it was in generator mode and vice versa).

In [ ]:
show_doc(GANTrainer.on_train_begin)

<h4 id="GANTrainer.on_train_begin"><code>on_train_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L90" class="source_link">[source]</a></h4>

> <code>on_train_begin</code>(<b>`kwargs`</b>)

Create the optimizers for the generator and critic if necessary, initialize smootheners.  

In [ ]:
show_doc(GANTrainer.on_epoch_begin)

<h4 id="GANTrainer.on_epoch_begin"><code>on_epoch_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L127" class="source_link">[source]</a></h4>

> <code>on_epoch_begin</code>(<b>`epoch`</b>, <b>`kwargs`</b>)

Put the critic or the generator back to eval if necessary.  

In [ ]:
show_doc(GANTrainer.on_batch_begin)

<h4 id="GANTrainer.on_batch_begin"><code>on_batch_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L110" class="source_link">[source]</a></h4>

> <code>on_batch_begin</code>(<b>`last_input`</b>, <b>`last_target`</b>, <b>`kwargs`</b>)

Clamp the weights with `self.clip` if it's not None, return the correct input.  

In [ ]:
show_doc(GANTrainer.on_backward_begin)

<h4 id="GANTrainer.on_backward_begin"><code>on_backward_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L116" class="source_link">[source]</a></h4>

> <code>on_backward_begin</code>(<b>`last_loss`</b>, <b>`last_output`</b>, <b>`kwargs`</b>)

Record `last_loss` in the proper list.  

In [ ]:
show_doc(GANTrainer.on_epoch_end)

<h4 id="GANTrainer.on_epoch_end"><code>on_epoch_end</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L131" class="source_link">[source]</a></h4>

> <code>on_epoch_end</code>(<b>`pbar`</b>, <b>`epoch`</b>, <b>`kwargs`</b>)

Put the various losses in the recorder and show a sample image.  

In [ ]:
show_doc(GANTrainer.on_train_end)

<h4 id="GANTrainer.on_train_end"><code>on_train_end</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L106" class="source_link">[source]</a></h4>

> <code>on_train_end</code>(<b>`kwargs`</b>)

Switch in generator mode for showing results.  

## Specific modules

In [ ]:
show_doc(GANModule, title_level=3)

<h3 id="GANModule"><code>class</code> <code>GANModule</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L41" class="source_link">[source]</a></h3>

> <code>GANModule</code>(<b>`generator`</b>:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)=<b><i>`None`</i></b>, <b>`critic`</b>:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)=<b><i>`None`</i></b>, <b>`gen_mode`</b>:`bool`=<b><i>`False`</i></b>) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Wrapper around a `generator` and a `critic` to create a GAN.  

If `gen_mode` is left as `None`, just put the model in the other mode (critic if it was in generator mode and vice versa).

In [ ]:
show_doc(GANModule.switch)

<h4 id="GANModule.switch"><code>switch</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L51" class="source_link">[source]</a></h4>

> <code>switch</code>(<b>`gen_mode`</b>:`bool`=<b><i>`None`</i></b>)

Put the model in generator mode if `gen_mode`, in critic mode otherwise.  

In [ ]:
show_doc(GANLoss, title_level=3)

<h3 id="GANLoss"><code>class</code> <code>GANLoss</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L55" class="source_link">[source]</a></h3>

> <code>GANLoss</code>(<b>`loss_funcG`</b>:`Callable`, <b>`loss_funcC`</b>:`Callable`, <b>`gan_model`</b>:[`GANModule`](/vision.gan.html#GANModule)) :: [`GANModule`](/vision.gan.html#GANModule)

Wrapper around `loss_funcC` (for the critic) and `loss_funcG` (for the generator).  

In [ ]:
show_doc(AdaptiveLoss, title_level=3)

<h3 id="AdaptiveLoss"><code>class</code> <code>AdaptiveLoss</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L290" class="source_link">[source]</a></h3>

> <code>AdaptiveLoss</code>(<b>`crit`</b>) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Expand the `target` to match the `output` size before applying `crit`.  

In [ ]:
show_doc(accuracy_thresh_expand)

<h4 id="accuracy_thresh_expand"><code>accuracy_thresh_expand</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L299" class="source_link">[source]</a></h4>

> <code>accuracy_thresh_expand</code>(<b>`y_pred`</b>:`Tensor`, <b>`y_true`</b>:`Tensor`, <b>`thresh`</b>:`float`=<b><i>`0.5`</i></b>, <b>`sigmoid`</b>:`bool`=<b><i>`True`</i></b>) → `Rank0Tensor`

Compute accuracy after expanding `y_true` to the size of `y_pred`.  

## Data Block API

In [ ]:
show_doc(NoisyItem, title_level=3)

<h3 id="NoisyItem"><code>class</code> <code>NoisyItem</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L229" class="source_link">[source]</a></h3>

> <code>NoisyItem</code>(<b>`noise_sz`</b>) :: [`ItemBase`](/core.html#ItemBase)

An random [`ItemBase`](/core.html#ItemBase) of size `noise_sz`.  

In [ ]:
show_doc(GANItemList, title_level=3)

<h3 id="GANItemList"><code>class</code> <code>GANItemList</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L235" class="source_link">[source]</a></h3>

> <code>GANItemList</code>(<b>`items`</b>, <b>`noise_sz`</b>:`int`=<b><i>`100`</i></b>, <b>`kwargs`</b>) :: [`ImageItemList`](/vision.data.html#ImageItemList)

`ItemList` suitable for GANs.  

Inputs will be [`NoisyItem`](/vision.gan.html#NoisyItem) of `noise_sz` while the default class for target is [`ImageItemList`](/vision.data.html#ImageItemList).

In [ ]:
show_doc(GANItemList.show_xys)

<h4 id="GANItemList.show_xys"><code>show_xys</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L247" class="source_link">[source]</a></h4>

> <code>show_xys</code>(<b>`xs`</b>, <b>`ys`</b>, <b>`imgsize`</b>:`int`=<b><i>`4`</i></b>, <b>`figsize`</b>:`Optional`\[`Tuple`\[`int`, `int`\]\]=<b><i>`None`</i></b>, <b>`kwargs`</b>)

Shows `ys` (target images) on a figure of `figsize`.  

In [ ]:
show_doc(GANItemList.show_xyzs)

<h4 id="GANItemList.show_xyzs"><code>show_xyzs</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L251" class="source_link">[source]</a></h4>

> <code>show_xyzs</code>(<b>`xs`</b>, <b>`ys`</b>, <b>`zs`</b>, <b>`imgsize`</b>:`int`=<b><i>`4`</i></b>, <b>`figsize`</b>:`Optional`\[`Tuple`\[`int`, `int`\]\]=<b><i>`None`</i></b>, <b>`kwargs`</b>)

Shows `zs` (generated images) on a figure of `figsize`.  

## Undocumented Methods - Methods moved below this line will intentionally be hidden

In [ ]:
show_doc(GANLoss.critic)

<h4 id="GANLoss.critic"><code>critic</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L66" class="source_link">[source]</a></h4>

> <code>critic</code>(<b>`real_pred`</b>, <b>`input`</b>)

Create some `fake_pred` with the generator from `input` and compare them to `real_pred` in `self.loss_funcD`.  

In [ ]:
show_doc(GANModule.forward)

<h4 id="GANModule.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L48" class="source_link">[source]</a></h4>

> <code>forward</code>(<b>`args`</b>)

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(GANLoss.generator)

<h4 id="GANLoss.generator"><code>generator</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L61" class="source_link">[source]</a></h4>

> <code>generator</code>(<b>`output`</b>, <b>`target`</b>)

Evaluate the `output` with the critic then uses `self.loss_funcG` to combine it with `target`.  

In [ ]:
show_doc(NoisyItem.apply_tfms)

<h4 id="NoisyItem.apply_tfms"><code>apply_tfms</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L233" class="source_link">[source]</a></h4>

> <code>apply_tfms</code>(<b>`tfms`</b>, <b>`kwargs`</b>)

Subclass this method if you want to apply data augmentation with `tfms` to this [`ItemBase`](/core.html#ItemBase).  

In [ ]:
show_doc(AdaptiveLoss.forward)

<h4 id="AdaptiveLoss.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L296" class="source_link">[source]</a></h4>

> <code>forward</code>(<b>`output`</b>, <b>`target`</b>)

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(GANItemList.get)

<h4 id="GANItemList.get"><code>get</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L244" class="source_link">[source]</a></h4>

> <code>get</code>(<b>`i`</b>)

Subclass if you want to customize how to create item `i` from `self.items`.  

In [ ]:
show_doc(GANItemList.reconstruct)

<h4 id="GANItemList.reconstruct"><code>reconstruct</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L245" class="source_link">[source]</a></h4>

> <code>reconstruct</code>(<b>`t`</b>)

Reconstuct one of the underlying item for its data `t`.  

In [ ]:
show_doc(AdaptiveLoss.forward)

<h4 id="AdaptiveLoss.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L296" class="source_link">[source]</a></h4>

> <code>forward</code>(<b>`output`</b>, <b>`target`</b>)

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

## New Methods - Please document or move to the undocumented section